# Regogida de datos para el proyecto Airevis

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

### Pasar los archivos a dataframe

In [ ]:
# Cargar el archivo .nc
dsContam = xr.open_dataset("data/2024AnalisisContam.nc")     # datos del 1 ene 2024 al 31 dic 2024 [CONTAMINACION]
dsPolem = xr.open_dataset("data/2024AnalisisPolem.nc")       # datos del 1 ene 2024 al 31 dic 2024 [POLEM]

# Convertir a DataFrame
dfContam = dsContam.to_dataframe().reset_index()
dfPolem = dsPolem.to_dataframe().reset_index()

C:\Users\alvar\AppData\Local\Temp\ipykernel_16928\2315334412.py:5: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsContam = xr.open_dataset("data/2024AnalisisContam.nc")     # datos del 1 ene 2024 al 31 dic 2024 [CONTAMINACION]
C:\Users\alvar\AppData\Local\Temp\ipykernel_16928\2315334412.py:6: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will 

In [2]:
dfContam.head()

,longitude,latitude,level,time,co_conc,o3_conc,pm10_conc,pm2p5_conc,so2_conc
0,357.049988,38.25,0.0,0 days 12:00:00,184.679382,52.928925,10.743207,8.305279,1.431829
1,357.049988,38.25,0.0,0 days 15:00:00,181.264877,62.529579,8.925305,5.089528,1.403990
2,357.049988,38.25,0.0,1 days 12:00:00,140.096985,70.810318,8.621214,3.987931,0.909928
3,357.049988,38.25,0.0,1 days 15:00:00,132.730423,71.771805,7.975132,3.061785,0.722260
4,357.049988,38.25,0.0,2 days 12:00:00,128.850906,66.330101,7.719180,3.959922,1.250328


In [3]:
dfPolem.head()

,longitude,latitude,level,time,gpg_conc,opg_conc
0,357.049988,38.25,0.0,0 days,0.000000e+00,1.380507e-29
1,357.049988,38.25,0.0,1 days,4.933570e-31,0.000000e+00
2,357.049988,38.25,0.0,2 days,8.301630e-29,0.000000e+00
3,357.049988,38.25,0.0,3 days,0.000000e+00,0.000000e+00
4,357.049988,38.25,0.0,4 days,0.000000e+00,0.000000e+00


### Arreglar el formato de fecha que venía en ambas tablas

In [ ]:
# arreglar time poniendole fecha
fecha_inicio = datetime(2024, 1, 1)
dfPolem["fecha"] = dfPolem["time"].apply(lambda x: fecha_inicio + x)

# renombrar columnas
dfPolem = dfPolem.rename(columns={
    "gpg_conc": "Polen Hierba (granos/m³)",
    "opg_conc": "Polen Olivos (granos/m³)"
})

# borrar lo innecesario
dfPolem = dfPolem.drop(columns=["longitude", "latitude", "level", "time"])

dfPolem.head()

,Polen Hierba (granos/m³),Polen Olivos (granos/m³),fecha
0,0.000000e+00,1.380507e-29,2024-01-01
1,4.933570e-31,0.000000e+00,2024-01-02
2,8.301630e-29,0.000000e+00,2024-01-03
3,0.000000e+00,0.000000e+00,2024-01-04
4,0.000000e+00,0.000000e+00,2024-01-05


In [ ]:
dfContam["datetime"] = dfContam["time"].apply(lambda x: fecha_inicio + x)

# separacion en fecha y hora
dfContam["fecha"] = dfContam["datetime"].dt.date
dfContam["hora"] = dfContam["datetime"].dt.time

# borrar lo innecesario
dfContam = dfContam.drop(columns=["time", "datetime"])

dfContam.head()

,longitude,latitude,level,co_conc,o3_conc,pm10_conc,pm2p5_conc,so2_conc,fecha,hora
0,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00
1,357.049988,38.25,0.0,181.264877,62.529579,8.925305,5.089528,1.403990,2024-01-01,15:00:00
2,357.049988,38.25,0.0,140.096985,70.810318,8.621214,3.987931,0.909928,2024-01-02,12:00:00
3,357.049988,38.25,0.0,132.730423,71.771805,7.975132,3.061785,0.722260,2024-01-02,15:00:00
4,357.049988,38.25,0.0,128.850906,66.330101,7.719180,3.959922,1.250328,2024-01-03,12:00:00


### Unión de las dos tablas

In [ ]:
# pasar a fecha normal
dfContam["fecha"] = pd.to_datetime(dfContam["fecha"])
dfPolem["fecha"] = pd.to_datetime(dfPolem["fecha"])

# union por fecha
dfTotal = pd.merge(dfContam, dfPolem, on="fecha", how="left")
dfTotal.head()

,longitude,latitude,level,co_conc,o3_conc,pm10_conc,pm2p5_conc,so2_conc,fecha,hora,Polen Hierba (granos/m³),Polen Olivos (granos/m³)
0,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.380507e-29
1,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,9.071900e-30
2,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,3.549874e-30
3,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.932709e-29
4,357.049988,38.25,0.0,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.301620e-29


### Renombrar y ordenar

In [ ]:
# level fuera ya que no importa
dfTotal = dfTotal.drop(columns=["level"])

dfTotal = dfTotal.rename(columns={
    "co_conc": "CO (µg/m³)",
    "o3_conc": "O₃ (µg/m³)",
    "pm10_conc": "PM10 (µg/m³)",
    "pm2p5_conc": "PM2.5 (µg/m³)",
    "so2_conc": "SO₂ (µg/m³)"
})

dfTotal.head()

,longitude,latitude,CO (µg/m³),O₃ (µg/m³),PM10 (µg/m³),PM2.5 (µg/m³),SO₂ (µg/m³),fecha,hora,Polen Hierba (granos/m³),Polen Olivos (granos/m³)
0,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.380507e-29
1,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,9.071900e-30
2,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,3.549874e-30
3,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.932709e-29
4,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,2024-01-01,12:00:00,0.0,1.301620e-29


In [19]:
columnas_ordenadas = [
    'fecha', 'hora', 'longitude', 'latitude',
    'CO (µg/m³)', 'O₃ (µg/m³)', 'PM10 (µg/m³)', 'PM2.5 (µg/m³)', 'SO₂ (µg/m³)',
    'Polen Hierba (granos/m³)', 'Polen Olivos (granos/m³)'
]

dfTotal = dfTotal[columnas_ordenadas]
dfTotal.head()

,fecha,hora,longitude,latitude,CO (µg/m³),O₃ (µg/m³),PM10 (µg/m³),PM2.5 (µg/m³),SO₂ (µg/m³),Polen Hierba (granos/m³),Polen Olivos (granos/m³)
0,2024-01-01,12:00:00,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,0.0,1.380507e-29
1,2024-01-01,12:00:00,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,0.0,9.071900e-30
2,2024-01-01,12:00:00,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,0.0,3.549874e-30
3,2024-01-01,12:00:00,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,0.0,1.932709e-29
4,2024-01-01,12:00:00,357.049988,38.25,184.679382,52.928925,10.743207,8.305279,1.431829,0.0,1.301620e-29


### Exportación

In [ ]:
dfTotal.to_csv("airevisDatos.csv", index=False)